In [1]:
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
import ta


In [2]:
# Fetch stock data
def fetch_stock_data(ticker, period, interval):
    end_date = datetime.now()
    if period == '1wk':
        start_date = end_date - timedelta(days=7)
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    else:
        data = yf.download(ticker, period=period, interval=interval)
    return data

# Process data to ensure it is in the right format
def process_data(data):
    if data.index.tzinfo is None:
        data.index = data.index.tz_localize('EAT')
    data.index = data.index.tz_convert('Kenya')
    data.reset_index(inplace=True)
    data.rename(columns={'Date': 'Datetime'}, inplace=True)
    return data

# Compute basic metrics from stock data
def calculate_metrics(data):
    last_close = data ['close'].iloc[-1]
    prev_close = data ['close'].iloc[0]
    change = last_close-prev_close
    pct_change = (change / last_close)*100
    High = data['High'].max()
    Low = data['Low'].min()
    Volume = data['Volume'].sum()
    
    return last_close, prev_close, change, pct_change, High, Low, Volume


In [3]:
# Add Simple Moving Average(SMA) and Eponential Moving Average(EMA) indicators 
def add_technical_indicators(data):
    data['SMA_20'] = ta.trend.sma_indicator(data['close'], window=20)
    data['EMA_20'] = ta.trend.sma_indicator(data['close'], window=20)
    return data

In [4]:
# set up streamlit layout page
st.set_page_config(layout='wide')
st.title('Real Time Stock Dashboard')

# Sidebar user input parameters
st.sidebar.header('Chart Parameters')
ticker = st.sidebar.text_input('Ticker', 'ADBE')
time_period = st.sidebar.selectbox('Time period', ['1d', '1wk', '1mo', '1y', 'max'])
chart_type = st.sidebar.selectbox('chart type', ['Candlestick', 'Line'])
indicators = st.sidebar.multiselect('Technical Indicators', ['SMA 20', 'EMA 20'])

# Mapping time periods to data intervals
interval_mapping = {
    '1d': '1m',
    '1wk': '30m',
    '1mo': '1d',
    '1y': '1wk',
    'max': '1wk'
}


2024-08-17 22:39:03.007 
  command:

    streamlit run C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-08-17 22:39:03.029 Session state does not function when running a script without `streamlit run`


In [5]:
# Update dashboard based on user input
if st.sidebar.button('Update'):
    data = fetch_stock_data(ticker, time_period, interval_mapping[time_period])
    data = process_data(data)
    data = add_technical_indicators(data)
    
    last_close, change, pct_change, high, low, volume = calculate_metrics(data)

# Display main metrics
st.metric(label=f"{ticker} Last Price", value=f"{last_close:.2f} USD", delta=f"{change:.2f} ({pct_change:.2f}%)")

col1, col2, col3=st.columns(3)
col1.metric("High", f"{high:.2f} USD")
col2.metric("Low", f"{low:.2f} USD")
col3.metric("volume", f"{volume:,}")

# Plot stock price chart
fig = go.Figure()
if chart_type == 'Candlestick':
    fig.add_trace(go.Candlestick(x=data['Datetime'],
                                 open=data['open'],
                                 high=data['high'],
                                 low=data['low'],
                                 close=data['Close']))
else:
    fig = px.line(data, x='Datetime', y='Close')

st.plotly_chart(fig, use_container_width=True)


NameError: name 'last_close' is not defined

In [7]:
# Add selected technical indicators to the chart
for indicator in indicators:
    if indicator == 'SMA 20':
        fig.add_trace(go.Scatter(x=data['Datetime'], y=data['SMA 20'], name='SMA 20'))
    elif indicator == 'EMA 20':
        fig.add_trace(go.Scatter(x=data['Datetime'], y=data['EMA 20'], name=['EMA 20']))

# Format graph
fig.update_layout(title=f'{ticker} {time_period.upper()} Chart',
                  xaxis_title='Time',
                  yaxis_title='Price (USD)',
                  height=600)
st.plotly_chart(fig, use_container_width=True)

NameError: name 'fig' is not defined

In [8]:
# Display historical data and technical indicators
st.subheader('Historical Data')
st.dataframe(data[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']])

st.subheader('Technical Indicators')
st.dataframe(data[['Datetime', 'SMA 20', 'EMA 20']])

NameError: name 'data' is not defined

In [9]:
# Real time stock prices 
st.sidebar.header('Real-Time Stock Prices')
stock_symbols = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
for symbol in stock_symbols:
    real_time_data = fetch_stock_data(symbol, '1d', '1m')
    if not real_time_data.empty:
        real_time_data = process_data(real_time_data)
        last_price = real_time_data['Close'].iloc[-1]
        change = last_price - real_time_data['Open'].iloc[0]
        pct_change = (change/real_time_data['Open'].iloc[0]) * 100
        st.bar.metric(f"{symbol}", f"{last_price} USD", f"{change:.2f} ({pct_change:.2f}%)")

# Information section
st.sidebar.subheader('About')
st.sidebar.info('This Dashboard Stock data and technical indicators for various time periods')

Failed to get ticker 'AAPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001924247A710>: Failed to resolve 'fc.yahoo.com' ([Errno 11001] getaddrinfo failed)"))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'fc.yahoo.com\', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019242EC79D0>: Failed to resolve \'fc.yahoo.com\' ([Errno 11001] getaddrinfo failed)"))'))
Failed to get ticker 'GOOGL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019242483D90>: Failed to resolve 'fc.yahoo.com' ([Errno 11001] getaddrinfo failed)"))
[*********************100

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())